In [2]:
import pandas as pd
pd.__version__

'1.1.1'

In [3]:
sGenes = pd.read_csv("sGenes.csv", header=0, index_col=0, quotechar='"', sep=",", na_values=["nan", "-", ".", " ", "", "NA"], delimiter=',')
#sGenes.head()

In [3]:
sGenes.shape

(11249, 24)

In [4]:
# I needed a file which has geneIDs to retrieve the homoeolog genes from Biomart
sGenes.GeneID.to_csv('sgenes.txt', index=False)
#for i in sGenes.GeneID.unique():
    #print(i)

In [5]:
sGenes.dtypes

intron                 object
AA                      int64
BB                      int64
A.pheno.mean          float64
B.pheno.mean          float64
effect                 object
intron_chr             object
intron_start            int64
intron_end              int64
qtl_chr                object
qtl_start(bp)         float64
qtl_end(bp)           float64
lod                   float64
intron_regions(bp)     object
qtl_regions(bp)        object
GeneID                 object
qtl_pos(cM)           float64
ci_lo(cM)             float64
ci_hi(cM)             float64
qtl_loc_pos            object
qtl_loc(ci_lo/hi)      object
clusterID              object
qtl_type               object
additive_effect       float64
dtype: object

In [6]:
sGenes.isnull().sum()

intron                   0
AA                       0
BB                       0
A.pheno.mean           484
B.pheno.mean             0
effect                 484
intron_chr               0
intron_start             0
intron_end               0
qtl_chr                  0
qtl_start(bp)         1104
qtl_end(bp)           1104
lod                      0
intron_regions(bp)       0
qtl_regions(bp)          0
GeneID                   0
qtl_pos(cM)              0
ci_lo(cM)                0
ci_hi(cM)                0
qtl_loc_pos              0
qtl_loc(ci_lo/hi)        0
clusterID                0
qtl_type               173
additive_effect        484
dtype: int64

In [33]:
sGenes = sGenes.fillna("NA")
#sGenes

In [8]:
print(sGenes.isnull().sum())

intron                0
AA                    0
BB                    0
A.pheno.mean          0
B.pheno.mean          0
effect                0
intron_chr            0
intron_start          0
intron_end            0
qtl_chr               0
qtl_start(bp)         0
qtl_end(bp)           0
lod                   0
intron_regions(bp)    0
qtl_regions(bp)       0
GeneID                0
qtl_pos(cM)           0
ci_lo(cM)             0
ci_hi(cM)             0
qtl_loc_pos           0
qtl_loc(ci_lo/hi)     0
clusterID             0
qtl_type              0
additive_effect       0
dtype: int64


In [9]:
DEGs = []
keys = sGenes.to_dict('records')

In [10]:
sGenes.columns

Index(['intron', 'AA', 'BB', 'A.pheno.mean', 'B.pheno.mean', 'effect',
       'intron_chr', 'intron_start', 'intron_end', 'qtl_chr', 'qtl_start(bp)',
       'qtl_end(bp)', 'lod', 'intron_regions(bp)', 'qtl_regions(bp)', 'GeneID',
       'qtl_pos(cM)', 'ci_lo(cM)', 'ci_hi(cM)', 'qtl_loc_pos',
       'qtl_loc(ci_lo/hi)', 'clusterID', 'qtl_type', 'additive_effect'],
      dtype='object')

In [4]:
sGenes_loc = sGenes[["GeneID", "qtl_loc(ci_lo/hi)"]]
#sGenes_loc

In [5]:
sGenes_loc.columns = ['Genes', 'location']
#sGenes_loc.drop_duplicates()

In [6]:
sqtl={}
for i in range(len(sGenes_loc.location.unique())): 
    keys = sGenes_loc.location.unique()
    subset = sGenes_loc.loc[sGenes_loc["location"] == keys[i]]
    values = subset["Genes"].tolist()
    values = set(values)
    sqtl[keys[i]] = values   

In [9]:
sqtl = dict(sorted(sqtl.items()))
#sqtl
#print(list(sqtl.keys())[11]) 
#list(sqtl.values())[11] 

In [10]:
#sqtl["2B:0:107.38738"]

In [11]:
#for x in sqtl["4B:0:0"]:
    #print(x)

In [12]:
#sqtl

In [13]:
homoeolog = pd.read_csv("homoeolog_genes_of_sGenes.txt")
homoeolog.dropna(subset = ["Triticum_aestivum_homoeologue_gene_stable_ID"], inplace=True)
print(homoeolog.shape)
#homoeolog.head(20)

(9909, 2)


In [14]:
print(len(set(homoeolog.Gene_stable_ID)))#5161, NA containing rows were dropped so there are 5161 genes
print(len(set(homoeolog.Triticum_aestivum_homoeologue_gene_stable_ID))) #8482

# check if there are homoelog genes in the sGenes
genes=[]
for x in set(homoeolog.Triticum_aestivum_homoeologue_gene_stable_ID):
    if x in set(homoeolog.Gene_stable_ID):
        genes.append(x)


        
print(len(genes))#2347

5161
8482
2347


In [16]:
all = {}
for k, v in sqtl.items(): 
    for i in v:
        subset = homoeolog.loc[homoeolog["Gene_stable_ID"] == i]
        #print(subset.shape)
        if subset.shape[0] > 0:
            look = subset.Triticum_aestivum_homoeologue_gene_stable_ID
            index = 0
            for y in look:
                rows = []
                #print(subset.Gene_stable_ID)
                if y[index] in set(subset.Gene_stable_ID):
                    result = y[index]
                    if len(result) > 0:
                        loc = k
                        rows.append([result])
                        all[loc]=rows
   


#print(all)

In [17]:
#df = pd.DataFrame(data, columns = ['colName1', 'ColName2']) 
rows = []
for k, v in sqtl.items():
    if len(v) > 1: 
        loc = k
        num = len(v)
        rows.append([loc, num])
        #print(loc, num)
        
df = pd.DataFrame(rows, columns=["location", "gene_number"])
#df.head(12)

In [21]:
df.shape

(831, 2)

In [18]:
df.sort_values(by=['gene_number'], inplace=True, ascending=False)
#df.head(20)

In [19]:
#df.iloc[100:105]

In [20]:
#df.tail(2)

In [21]:
#df[df.values == 3 ]
#df[df.values == 6 ]

In [22]:
#for i in sqtl["6B:111:111"]:
    #print(i)

In [23]:
#for i in sqtl["4B:2:3"]:
    #print(i)


In [24]:
B6_111_111 = pd.read_csv("6B_111_111.txt")
#B6_111_111

In [25]:
print(B6_111_111.shape)
len(B6_111_111.Gene_stable_ID.unique())

(191, 2)


97

In [27]:
p_rows = []
n_rows = []
for i in B6_111_111.Triticum_aestivum_homoeologue_gene_stable_ID:
    if i in B6_111_111.Gene_stable_ID:
        p_gene = i
        p_rows.append([p_gene])
    else:
        n_gene = i       
        n_rows.append([n_gene])

#p_rows
#n_rows
#print(len(p_rows))
#len(n_rows)

In [28]:
# repeat the codes 
#for i in sqtl["2B:106.97984:107.38738"]:
    #print(i)

In [29]:
B2_106_107 = pd.read_csv("2B_106.97984_107.38738.txt")
#B2_106_107

In [30]:
print(B2_106_107.shape)
len(B2_106_107.Gene_stable_ID.unique())

(260, 2)


136

In [32]:
p_rows = []
n_rows = []
for i in B2_106_107.Triticum_aestivum_homoeologue_gene_stable_ID:
    if i in B2_106_107.Gene_stable_ID:
        p_gene = i
        p_rows.append([p_gene])
    else:
        n_gene = i       
        n_rows.append([n_gene])

print(len(p_rows))
len(n_rows)

0


260

In [ ]:
import matplotlib.pyplot as plt
df.plot(kind='scatter',x='location',y='gene_number',color='red')
#plt.show()

<AxesSubplot:xlabel='location', ylabel='gene_number'>